# VOLCO FEA Module Demo

This notebook demonstrates the Finite Element Analysis (FEA) capabilities of the VOLCO package. It shows:

1. Running a VOLCO simulation to generate a voxel model
2. Applying basic boundary conditions for structural analysis
3. Using expert boundary conditions for the same setup
4. Exporting and importing FEA results

All examples use a standard 1% compression test.

## 1. Setup and Imports

In [ ]:
import os
import numpy as np
import sys

# Add the parent directory to the path if running from examples directory
sys.path.append(os.path.abspath('..'))

# Import VOLCO and VOLCO FEA modules
from volco import run_simulation
from volco_fea import (
    analyze_voxel_matrix, 
    Surface, 
    visualize_fea, 
    export_visualization,
    load_fea_results,
    select_nodes_by_predicate
)

## 2. Run VOLCO Simulation

First, we'll run a VOLCO simulation to generate a voxel model. We'll use this model for all FEA examples.

In [ ]:
# Create output directory if it doesn't exist
os.makedirs("Results_volco/fea", exist_ok=True)

# Run VOLCO simulation
print("Running VOLCO simulation...")
output = run_simulation(
    gcode_path='gcode_example.gcode',
    printer_config_path='printer_settings.json',
    sim_config_path='simulation_settings.json'
)

# Get the cropped voxel matrix from the simulation output
voxel_matrix = output.cropped_voxel_space
voxel_size = output._simulation.voxel_size

print(f"Simulation complete. Cropped voxel matrix shape: {voxel_matrix.shape}")
print(f"Voxel size: {voxel_size} mm")

## 3. Basic Boundary Conditions Example

Now we'll run an FEA analysis using basic boundary conditions. We'll apply a 1% compression to the model by:
- Fixing the bottom surface (MINUS_Z)
- Applying a downward displacement to the top surface (PLUS_Z)

In [ ]:
# Define material properties for PLA
material_properties = {
    'young_modulus': 2000.0,  # MPa (typical for PLA)
    'poisson_ratio': 0.3      # Typical for PLA
}

# Calculate model height and displacement magnitude (1% of model height)
model_height = voxel_matrix.shape[2] * voxel_size
displacement_magnitude = model_height * 0.01

print(f"Model height: {model_height:.3f} mm")
print(f"Applying displacement of {displacement_magnitude:.6f} mm (1% of model height)")

# Define boundary conditions using Simple Mode
basic_boundary_conditions = {
    'constraints': {
        Surface.MINUS_Z: "fix",  # Fix bottom surface
        Surface.PLUS_Z: [None, None, -displacement_magnitude, None, None, None]  # Apply 1% compression on top
    }
}

# Run the analysis with basic boundary conditions
print("\nRunning FEA analysis with basic boundary conditions...")
basic_results = analyze_voxel_matrix(
    voxel_matrix=voxel_matrix,
    voxel_size=voxel_size,
    material_properties=material_properties,
    boundary_conditions=basic_boundary_conditions,
    visualization=True,
    result_type='von_mises',
    scale_factor=10.0  # Exaggerate deformation for visualization
)

print("Basic boundary conditions analysis complete.")
print(f"Maximum displacement: {basic_results['max_displacement']:.6f} mm")
print(f"Maximum von Mises stress: {basic_results['max_von_mises']:.2f} MPa")

### Visualize Basic Boundary Conditions Results

In [ ]:
# Display the von Mises stress visualization and/or save it to file
if 'visualization' in basic_results:
    basic_results['visualization'].show()
    
    # Save the visualization to file
    export_visualization(
        basic_results['visualization'],
        "Results_volco/fea/basic_boundary_conditions.html"
    )
    print("Visualization saved to Results_volco/fea/basic_boundary_conditions.html")

## 4. Expert Boundary Conditions Example

Now we'll run the same analysis but using expert boundary conditions. This gives more control over how constraints are applied.

In [ ]:
# Define custom boundary conditions using expert mode
def custom_constraint_function(nodes, elements):
    # Get model dimensions directly from nodes
    z_coords = nodes[:, 2]
    min_z = np.min(z_coords)
    max_z = np.max(z_coords)
    
    # Calculate model height
    model_height = max_z - min_z
    
    # Calculate displacement as 1% of model height (same as basic example)
    displacement_magnitude = model_height * 0.01
    
    # Create constraints dictionary
    constraints = {}
    
    # Fix nodes on the bottom surface (within a small tolerance)
    tolerance = 1e-6
    for i in range(len(nodes)):
        if abs(nodes[i, 2] - min_z) < tolerance:
            constraints[i] = [0, 0, 0, 0, 0, 0]  # Fix all DOFs
    
    # Apply displacement to nodes on the top surface
    for i in range(len(nodes)):
        if abs(nodes[i, 2] - max_z) < tolerance:
            constraints[i] = [None, None, -displacement_magnitude, None, None, None]
    
    return constraints

# Define expert boundary conditions
expert_boundary_conditions = {
    'constraints': {
        "custom": custom_constraint_function
    }
}

# Run the analysis with expert boundary conditions
print("\nRunning FEA analysis with expert boundary conditions...")
expert_results = analyze_voxel_matrix(
    voxel_matrix=voxel_matrix,
    voxel_size=voxel_size,
    material_properties=material_properties,
    boundary_conditions=expert_boundary_conditions,
    visualization=True,
    result_type='von_mises',
    scale_factor=10.0  # Exaggerate deformation for visualization
)

print("Expert boundary conditions analysis complete.")
print(f"Maximum displacement: {expert_results['max_displacement']:.6f} mm")
print(f"Maximum von Mises stress: {expert_results['max_von_mises']:.2f} MPa")

### Visualize Expert Boundary Conditions Results

In [ ]:
# Display the von Mises stress visualization
if 'visualization' in expert_results:
    expert_results['visualization'].show()

## 5. Compare Basic and Expert Results

Let's compare the results from both approaches to verify they produce similar outcomes.

In [ ]:
# Compare maximum displacement and stress
print("Comparison of Basic vs Expert Boundary Conditions:")
print(f"Basic max displacement: {basic_results['max_displacement']:.6f} mm")
print(f"Expert max displacement: {expert_results['max_displacement']:.6f} mm")
print(f"Difference: {abs(basic_results['max_displacement'] - expert_results['max_displacement']):.6f} mm")
print()
print(f"Basic max von Mises stress: {basic_results['max_von_mises']:.2f} MPa")
print(f"Expert max von Mises stress: {expert_results['max_von_mises']:.2f} MPa")
print(f"Difference: {abs(basic_results['max_von_mises'] - expert_results['max_von_mises']):.2f} MPa")

## 6. Export Results to File

Now we'll demonstrate how to save FEA results to a file for later use.

In [ ]:
# Run analysis and save results to file
print("Running FEA analysis and saving results to file...")
saved_results = analyze_voxel_matrix(
    voxel_matrix=voxel_matrix,
    voxel_size=voxel_size,
    material_properties=material_properties,
    boundary_conditions=basic_boundary_conditions,  # Using basic boundary conditions
    visualization=True,
    result_type='von_mises',
    scale_factor=10.0,
    save_results=True,
    save_path='Results_volco/fea/demo_results',
    save_format='pickle'
)

print(f"Results saved to: {saved_results['saved_file']}")
print(f"Maximum displacement: {saved_results['max_displacement']:.6f} mm")
print(f"Maximum von Mises stress: {saved_results['max_von_mises']:.2f} MPa")

## 7. Import Results from File

Now we'll demonstrate how to load the saved results and visualize them.

In [ ]:
# Load results from file
print("Loading FEA results from file...")
loaded_results = load_fea_results('Results_volco/fea/demo_results.pkl')

# Display key information from the loaded results
print("\nLoaded FEA Results Summary:")
print(f"Number of nodes: {loaded_results['nodes'].shape[0]}")
print(f"Number of elements: {loaded_results['elements'].shape[0]}")
print(f"Maximum displacement: {loaded_results['max_displacement']:.6f} mm")
print(f"Maximum von Mises stress: {loaded_results['max_von_mises']:.2f} MPa")

### Create Visualization from Loaded Results

In [ ]:
# Create visualization from loaded results
print("Creating visualization from loaded results...")
viz = visualize_fea(
    nodes=loaded_results['nodes'],
    elements=loaded_results['elements'],
    displacements=loaded_results['displacements'],
    von_mises=loaded_results['von_mises'],
    result_type='von_mises',
    scale_factor=10.0,
    show_undeformed=True,  # Show both original and deformed meshes
    original_opacity=0.3   # Set transparency of original mesh
)

# Display the visualization
viz.show()

## 8. Create Displacement Visualization

Finally, let's create a visualization showing the displacement magnitude instead of stress.

In [ ]:
# Create displacement visualization
print("Creating displacement visualization...")
disp_viz = visualize_fea(
    nodes=loaded_results['nodes'],
    elements=loaded_results['elements'],
    displacements=loaded_results['displacements'],
    von_mises=loaded_results['von_mises'],
    result_type='displacement',
    scale_factor=10.0,
    show_undeformed=False
)

# Display the visualization
disp_viz.show()

## Summary

In this notebook, we've demonstrated:

1. Running a VOLCO simulation to generate a voxel model
2. Applying basic boundary conditions for a 1% compression test
3. Using expert boundary conditions for the same test
4. Exporting FEA results to a file
5. Importing FEA results from a file
6. Creating different visualizations of the results

The results show that both basic and expert boundary conditions produce similar results for this simple compression test, but the expert mode offers more flexibility for complex scenarios.